In [1]:
# Importing the libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [20]:
# Importing the dataset
dataset = pd.read_csv('Churn_Modelling.csv')
X = dataset.iloc[:, 3:13].values
y = dataset.iloc[:, 13].values


# Encoding categorical data
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer

labelencoder_X_1 = LabelEncoder()
X[:, 1] = labelencoder_X_1.fit_transform(X[:, 1])
labelencoder_X_2 = LabelEncoder()
X[:, 2] = labelencoder_X_2.fit_transform(X[:, 2])

ct = ColumnTransformer(
    [('one_hot_encoder', OneHotEncoder(categories='auto'), [1])],
    remainder='passthrough',
)

X = np.array(ct.fit_transform(X), dtype=np.float)
X = X[:, 1:]

In [45]:
# Splitting the dataset into the Training set and Test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

# Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [46]:
# Importing the Keras libraries and packages
import keras
from keras.models import Sequential
from keras.layers import Dense

Using TensorFlow backend.


In [47]:
# Initialising the ANN
classifier = Sequential()

# Adding the input layer and the first hidden layer
classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu', input_dim = 11))

# Adding the second hidden layer
classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu'))

# Adding the output layer
classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))

# Compiling the ANN
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

# Fitting the ANN to the Training set
classifier.fit(X_train, y_train, batch_size = 10, epochs = 100)

Epoch 1/100
8000/8000 [==============================] - 3s 344us/step - loss: 0.4967 - accuracy: 0.7952
Epoch 2/100
8000/8000 [==============================] - 2s 260us/step - loss: 0.4304 - accuracy: 0.7960
Epoch 3/100
8000/8000 [==============================] - 2s 265us/step - loss: 0.4268 - accuracy: 0.7960
Epoch 4/100
8000/8000 [==============================] - 2s 254us/step - loss: 0.4230 - accuracy: 0.7959
Epoch 5/100
8000/8000 [==============================] - 2s 249us/step - loss: 0.4192 - accuracy: 0.8209
Epoch 6/100
8000/8000 [==============================] - 2s 278us/step - loss: 0.4165 - accuracy: 0.8260
Epoch 7/100
8000/8000 [==============================] - 2s 248us/step - loss: 0.4152 - accuracy: 0.8273
Epoch 8/100
8000/8000 [==============================] - 2s 255us/step - loss: 0.4133 - accuracy: 0.8306
Epoch 9/100
8000/8000 [==============================] - 2s 274us/step - loss: 0.4123 - accuracy: 0.8319
Epoch 10/100
8000/8000 [==============================]

In [51]:
# Predicting the Test set results
y_pred = classifier.predict(X_test)
y_pred = (y_pred > 0.5)

# Making the Confusion Matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)

In [52]:
# Predicting a single new observation
"""Predict if the customer with the following informations will leave the bank:
Geography: France
Credit Score: 600
Gender: Male
Age: 40
Tenure: 3
Balance: 60000
Number of Products: 2
Has Credit Card: Yes
Is Active Member: Yes
Estimated Salary: 50000"""
new_prediction = classifier.predict(sc.transform(np.array([[0.0, 0, 600, 1, 40, 3, 60000, 2, 1, 1, 50000]])))
new_prediction = (new_prediction > 0.5)

In [53]:
# Evaluating the ANN
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from keras.models import Sequential
from keras.layers import Dense
def build_classifier():
    classifier = Sequential()
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu', input_dim = 11))
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu'))
    classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))
    classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
    return classifier
classifier = KerasClassifier(build_fn = build_classifier, batch_size = 10, epochs = 100)
accuracies = cross_val_score(estimator = classifier, X = X_train, y = y_train, cv = 10, n_jobs = -1)
mean = accuracies.mean()
variance = accuracies.std()

In [54]:
# Tuning the ANN
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV
from keras.models import Sequential
from keras.layers import Dense
def build_classifier(optimizer):
    classifier = Sequential()
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu', input_dim = 11))
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu'))
    classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))
    classifier.compile(optimizer = optimizer, loss = 'binary_crossentropy', metrics = ['accuracy'])
    return classifier
classifier = KerasClassifier(build_fn = build_classifier)
parameters = {'batch_size': [25, 32],
              'epochs': [100, 500],
              'optimizer': ['adam', 'rmsprop']}
grid_search = GridSearchCV(estimator = classifier,
                           param_grid = parameters,
                           scoring = 'accuracy',
                           cv = 10)
grid_search = grid_search.fit(X_train, y_train)
best_parameters = grid_search.best_params_
best_accuracy = grid_search.best_score_

=====================] - 0s 64us/step - loss: 0.3900 - accuracy: 0.8361
Epoch 66/100
7200/7200 [==============================] - 0s 65us/step - loss: 0.3898 - accuracy: 0.8365
Epoch 67/100
7200/7200 [==============================] - 0s 63us/step - loss: 0.3900 - accuracy: 0.8376
Epoch 68/100
7200/7200 [==============================] - 0s 63us/step - loss: 0.3900 - accuracy: 0.8397
Epoch 69/100
7200/7200 [==============================] - 0s 63us/step - loss: 0.3900 - accuracy: 0.8372
Epoch 70/100
7200/7200 [==============================] - 0s 63us/step - loss: 0.3901 - accuracy: 0.8394
Epoch 71/100
7200/7200 [==============================] - 0s 64us/step - loss: 0.3898 - accuracy: 0.8394
Epoch 72/100
7200/7200 [==============================] - 0s 65us/step - loss: 0.3900 - accuracy: 0.8371
Epoch 73/100
7200/7200 [==============================] - 0s 65us/step - loss: 0.3898 - accuracy: 0.8388
Epoch 74/100
7200/7200 [==============================] - 0s 64us/step - loss: 0.3897 - 